In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import requests
import json

stocks_path = "/Data/stocks.csv"
commodities_path = "/Data/commodities.csv"

save_path = "/Data/"

In [ ]:
stocks_df = pd.read_csv(stocks_path)
stocks_df['CF_ID'] = stocks_df['CF_ID'].astype(int)

commodities_df = pd.read_csv(commodities_path)
commodities_df['CF_ID'] = commodities_df['CF_ID'].astype(int)


In [ ]:
stocks_df.head()

,Company,Ticker,CF_ID
0,Exxon Mobil,XOM,22
1,Chevron,CVX,21
2,ConocoPhillips,COP,186
3,EOG Resources,EOG,214
4,Schlumberger,SLB,177


In [ ]:
commodities_df.head()

,Name,Symbol,CF_ID
0,Gold,GCUSD,50
1,Silver,SIUSD,61
2,Copper,HGUSD,3685
3,Platinum,PLUSD,3686
4,Aluminum,ALIUSD,5500


In [ ]:
def get_sentiment_data(id):
  payload = {
            'api_key': '',
            'period': '',
            'entity_type': '',
            'duration': '',
            # # 'entity_name': entity_name,
            'entity_id': f'{id}'
            }
  response = requests.get('/entity_chart_data',
                              params=payload)
  return response.json(), response.url

In [ ]:
sentiment_df = pd.DataFrame()
confidence_df = pd.DataFrame()

for index, row in stocks_df.iterrows():
    id = row['CF_ID']
    ticker = row['Ticker']
    sentiment_data, url = get_sentiment_data(id)
    print(url)

    if sentiment_data:
        labels = sentiment_data['labels']
        values = sentiment_data['values']
        confidence = sentiment_data['confidence']

        labels = [pd.to_datetime(label, format='%d.%m.%Y').strftime('%Y-%m-%d') for label in labels]

        sentiment_df[ticker] = pd.Series(values, index=labels)
        confidence_df[ticker] = pd.Series(confidence, index=labels)

# Transpose the DataFrames to have companies as rows and timestamps as columns
sentiment_df = sentiment_df.transpose().rename_axis('Ticker')
confidence_df = confidence_df.transpose().rename_axis('Ticker')

In [ ]:
sentiment_df.head()

,2020-06-01,2020-06-02,2020-06-03,2020-06-04,2020-06-05,2020-06-06,2020-06-07,2020-06-08,2020-06-09,2020-06-10,...,2024-07-07,2024-07-08,2024-07-09,2024-07-10,2024-07-11,2024-07-12,2024-07-13,2024-07-14,2024-07-15,2024-07-16
Ticker,,,,,,,,,,,,,,,,,,,,,
XOM,-3.0,-20.0,11.0,-10.0,21.0,-18.0,20.0,-22.0,-27.0,-11.0,...,0.0,-10.0,-16.0,1.0,-5.0,-23.0,25.0,11.0,5.0,48.0
CVX,-27.0,NaN,-93.0,-17.0,-29.0,-68.0,15.0,15.0,1.0,-18.0,...,-21.0,-33.0,-27.0,-25.0,-38.0,-11.0,-18.0,-5.0,-15.0,-16.0
COP,-33.0,NaN,NaN,1.0,-85.0,NaN,NaN,-9.0,-9.0,4.0,...,NaN,-72.0,26.0,-49.0,34.0,11.0,-27.0,-100.0,4.0,100.0
EOG,-83.0,42.0,4.0,-51.0,-93.0,-44.0,NaN,20.0,-40.0,50.0,...,NaN,NaN,53.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SLB,NaN,-50.0,NaN,-10.0,54.0,NaN,NaN,86.0,22.0,NaN,...,0.0,0.0,0.0,52.0,67.0,26.0,NaN,NaN,-63.0,-6.0


In [ ]:
sentiment_df_commod = pd.DataFrame()
confidence_df_commod = pd.DataFrame()

for index, row in commodities_df.iterrows():
    id = row['CF_ID']
    ticker = row['Symbol']
    sentiment_data, url = get_sentiment_data(id)
    print(url)

    if sentiment_data:
        labels = sentiment_data['labels']
        values = sentiment_data['values']
        confidence = sentiment_data['confidence']

        labels = [pd.to_datetime(label, format='%d.%m.%Y').strftime('%Y-%m-%d') for label in labels]

        sentiment_df_commod[ticker] = pd.Series(values, index=labels)
        confidence_df_commod[ticker] = pd.Series(confidence, index=labels)

# Transpose the DataFrames to have companies as rows and timestamps as columns
sentiment_df_commod = sentiment_df_commod.transpose().rename_axis('Ticker')
confidence_df_commod = confidence_df_commod.transpose().rename_axis('Ticker')

In [ ]:
sentiment_df_commod

,2020-06-01,2020-06-02,2020-06-03,2020-06-04,2020-06-05,2020-06-06,2020-06-07,2020-06-08,2020-06-09,2020-06-10,...,2024-07-07,2024-07-08,2024-07-09,2024-07-10,2024-07-11,2024-07-12,2024-07-13,2024-07-14,2024-07-15,2024-07-16
Ticker,,,,,,,,,,,,,,,,,,,,,
GCUSD,20.0,12.0,11.0,13.0,2.0,2.0,6.0,19.0,17.0,24.0,...,19.0,-4.0,15.0,21.0,28.0,23.0,18.0,15.0,17.0,26.0
SIUSD,26.0,10.0,-4.0,22.0,1.0,6.0,13.0,16.0,3.0,9.0,...,27.0,13.0,31.0,15.0,23.0,-1.0,12.0,24.0,6.0,16.0
HGUSD,23.0,11.0,5.0,26.0,14.0,-11.0,-11.0,19.0,15.0,31.0,...,4.0,9.0,13.0,6.0,23.0,-10.0,25.0,-25.0,22.0,2.0
PLUSD,5.0,NaN,42.0,22.0,21.0,-30.0,7.0,29.0,36.0,33.0,...,0.0,62.0,18.0,41.0,69.0,7.0,79.0,75.0,35.0,-6.0
ALIUSD,32.0,-15.0,-10.0,56.0,-4.0,-35.0,-24.0,9.0,-4.0,-2.0,...,32.0,14.0,42.0,15.0,22.0,11.0,-4.0,-37.0,-5.0,-13.0
CLUSD,-52.0,-23.0,-39.0,-37.0,-24.0,-31.0,-31.0,-26.0,-36.0,-42.0,...,33.0,-51.0,-29.0,-13.0,20.0,-6.0,-52.0,-9.0,-20.0,-51.0
NGUSD,58.0,22.0,21.0,27.0,-20.0,-54.0,-46.0,-24.0,-20.0,-30.0,...,-17.0,3.0,4.0,3.0,7.0,9.0,24.0,4.0,5.0,14.0
PAUSD,56.0,NaN,52.0,NaN,23.0,NaN,99.0,27.0,-15.0,33.0,...,57.0,-5.0,83.0,33.0,27.0,-25.0,-24.0,-50.0,18.0,-22.0
